# GPU: CuPy, Numba-GPU, PyCUDA

If you can get better memory efficiency using rowwise code (e.g. compiled for loops), why would you ever write columnar code (e.g. Numpy)?

**Answer:** vectorization!

Vectorization is a vertical scaling technique that uses a single CPU core or a GPU more effectively. You can compute N operations at the same time _if they are all the same operation._

<center><img src="img/vectorization-example.png" width="50%"></center>

If you don't fully utilize all cores, that's okay; someone else's work can fill the gaps.

If you don't fully utilize the core's vector unit, no one else can use them.

A GPU is a computational device designed around vector units.

Like parallel processing, this is another computing detail that is visible to you as a data analyst. Rowwise code like

```python
@numba.jit
def run_numba_loop(height, width, maxiterations, c, fractal):
    for h in range(height):
        for w in range(width):
            z = c[h, w]
            for i in range(maxiterations):
                z = z**2 + c[h, w]
                if abs(z) > 2:
                    fractal[h, w] = i
                    break
    return fractal
```

does not use vector units effectively because each array element may be in a different stage of processing— some may have diverged before others.

Columnar code like

In [1]:
import numpy
import time

def prepare(height, width):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32)
    return c, fractal

def run(c, fractal, maxiterations=20):
    fractal *= 0                  # set fractal to maxiterations without replacing it
    fractal += maxiterations
    z = c
    for i in range(maxiterations):
        z = z**2 + c
        diverge = z.real**2 + z.imag**2 > 2**2
        divnow = diverge & (fractal == maxiterations)
        fractal[divnow] = i
        z[diverge] = 2
    return fractal

can make effective use of vector units because it's always applying the <b>S</b>ame <b>I</b>nstruction on <b>M</b>ultiple <b>D</b>ata (SIMD).

All we need is a librrary to implement the Numpy functions on a GPU.

In [2]:
import cupy

In [3]:
c, fractal = prepare(4000, 6000)

c = cupy.array(c)
fractal = cupy.array(fractal)

starttime = time.time()
fractal = run(c, fractal)
time.time() - starttime

2.889005661010742

In [4]:
c, fractal = prepare(4000, 6000)

starttime = time.time()
fractal = run(c, fractal)
time.time() - starttime

7.4934611320495605

Exactly the same code: first with CuPy on the GPU (2.8 sec), then with Numpy on the CPU (7.5 sec).

If you're wondering why I'm working on a reduced problem (4 time smaller than previous sessions), it's because I couldn't fit the full one in my GPU's memory!

(There's always a catch!)

Also, CuPy's adherence to the Numpy API isn't perfect: I had to write

```python
z.real**2 + z.imag**2
```

instead of

```python
numpy.absolute(z)
```

because the `absolute` function wasn't supported. This is the error you'd see:

In [6]:
try:
    numpy.absolute(cupy.array([1.1, 2.2, 3.3]))
except ValueError as err:
    print(err)

object __array__ method not producing an array


Nevertheless, we can expect CuPy to become more complete as people use it and report bugs.

**GPU method #2:** Use Numba! (You have to install a "cudatoolkit" library with it.)

In [13]:
import numba.cuda

@numba.cuda.jit
def as_cuda(c, fractal, maxiterations):
    x, y = numba.cuda.grid(2)     # 2 dimensional CUDA grid
    z = c[x, y]
    for i in range(maxiterations):
        z = z**2 + c[x, y]
        if abs(z) > 2:
            fractal[x, y] = i
            break                 # not optimal: threads that leave the loop still have to wait

def run_numba(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    return as_cuda(c, fractal, maxiterations)

In [12]:
starttime = time.time()
fractal = run_numba(4000, 6000)
time.time() - starttime

0.49817538261413574

On the same sized problem,

   * Numpy on the CPU: 7.5 sec
   * CuPy on the GPU: 2.8 sec
   * Numba on the GPU: 0.5 sec

And Numba doesn't suffer from the memory issue because it doesn't make as many intermediate copies.

In [14]:
starttime = time.time()
fractal = run_numba(8000, 12000)    # full-sized problem
time.time() - starttime

1.278907060623169